In [1]:
import os 
from openai import OpenAI
from dotenv import load_dotenv
import gradio as gr
import json

/Users/saud/Desktop/LLM_Projects/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key and openai_api_key.startswith('sk-proj-'):
    print('API Key exists, good to go')
else:
    print('API Key doesnt exist, troubleshoot the problem!')

API Key exists, good to go


In [3]:
openai = OpenAI()
MODEL = 'gpt-4o-mini'

system_prompt =  "You are a helpful assistant for an Airline called FlighAI. "
system_prompt += "Give short, courteous answers, not more than 1 sentence. "
system_prompt += "Always be accurate. If you don't know the answer, say so."


In [4]:
def get_ticket_prices(destination_city):
    ticket_prices = {'tokyo': '1499', 'paris': '599', 'sydney': '499', 'berlin': '799'}
    city = destination_city.lower()
    return ticket_prices.get(city, 'Unknown')

In [5]:
price_tool_function = {
    "name": "get_ticket_prices",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to."
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [6]:
tool = [{'type': 'function', 'function': price_tool_function}]

In [7]:
def handle_tool_call(assistant_message):
    tool_call = assistant_message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_prices(city)
    tool_response = {
        'role': 'tool',
        'content': json.dumps({"destination_city": city, "price": price}),
        'tool_call_id': tool_call.id
    }
    return tool_response, city

In [10]:
def airline_customer_support_agent(user_prompt, history):
    messages = [{'role': 'system', 'content': system_prompt}] + history + [{'role': 'user', 'content': user_prompt}]
    response = openai.chat.completions.create(
        model= MODEL,
        messages= messages,
        temperature= 0.5,
        tools= tool
    )
    print('History is:')
    print(history)
    print('And message is:')
    print(messages)

    if response.choices[0].finish_reason == 'tool_calls':
        assistant_response = response.choices[0].message
        tool_response,city = handle_tool_call(assistant_response)
        messages.append(assistant_response)
        print('after appending the assistant response: ')
        print(messages)
        messages.append(tool_response)
        print('after appending the tool response: ')
        print(messages)
        response = openai.chat.completions.create(model=MODEL, messages=messages)

    
    return response.choices[0].message.content

In [11]:
gr.ChatInterface(fn= airline_customer_support_agent, type= 'messages').launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


History is:
[]
And message is:
[{'role': 'system', 'content': "You are a helpful assistant for an Airline called FlighAI. Give short, courteous answers, not more than 1 sentence. Always be accurate. If you don't know the answer, say so."}, {'role': 'user', 'content': 'Hi'}]
History is:
[{'role': 'user', 'metadata': {'title': None}, 'content': 'Hi'}, {'role': 'assistant', 'metadata': {'title': None}, 'content': 'Hello! How can I assist you today?'}]
And message is:
[{'role': 'system', 'content': "You are a helpful assistant for an Airline called FlighAI. Give short, courteous answers, not more than 1 sentence. Always be accurate. If you don't know the answer, say so."}, {'role': 'user', 'metadata': {'title': None}, 'content': 'Hi'}, {'role': 'assistant', 'metadata': {'title': None}, 'content': 'Hello! How can I assist you today?'}, {'role': 'user', 'content': 'what is price of a return ticket?'}]
History is:
[{'role': 'user', 'metadata': {'title': None}, 'content': 'Hi'}, {'role': 'assi